In [1]:
!pip install textsearch
!pip install contractions
import nltk
nltk.download('punkt')
nltk.download('stopwords')

     |████████████████████████████████| 321 kB 5.8 MB/s 
     |████████████████████████████████| 283 kB 49.6 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85452 sha256=f09fe59a91214423d8256913a534ac264215511a1e6aa4a78ee80c035d446b32
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [2]:
import pandas as pd

df = pd.read_csv('https://github.com/dipanjanS/nlp_workshop_dhs18/raw/master/Unit%2010%20-%20Project%208%20-%20Movie%20Recommendations%20with%20Document%20Similarity/tmdb_5000_movies.csv.gz', compression='gzip')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   id                    4803 non-null   int64  
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [3]:
df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [4]:
df = df[['title', 'tagline', 'overview', 'popularity']]
df.tagline.fillna('', inplace=True)
df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df = df.sort_values(by=['popularity'], ascending=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4800 entries, 546 to 4553
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   title        4800 non-null   object 
 1   tagline      4800 non-null   object 
 2   overview     4800 non-null   object 
 3   popularity   4800 non-null   float64
 4   description  4800 non-null   object 
dtypes: float64(1), object(4)
memory usage: 225.0+ KB


In [5]:

import nltk
import re
import numpy as np
import contractions

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    doc = contractions.fix(doc)
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    #filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['description']))
len(norm_corpus)

4800

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(ngram_range= (1,2),min_df = 2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix.shape

(4800, 20471)

In [8]:
#pairwise Document similarity 
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,4760,4761,4762,4763,4764,4765,4766,4767,4768,4769,4770,4771,4772,4773,4774,4775,4776,4777,4778,4779,4780,4781,4782,4783,4784,4785,4786,4787,4788,4789,4790,4791,4792,4793,4794,4795,4796,4797,4798,4799
0,1.000000,0.000000,0.000000,0.000000,0.006071,0.008067,0.0,0.000000,0.0,0.000000,0.025531,0.008554,0.018111,0.000000,0.000000,0.000000,0.000000,0.007439,0.010454,0.000000,0.000000,0.008190,0.008365,0.010035,0.000000,0.000000,0.050976,0.006502,0.0,0.010728,0.0,0.006908,0.000000,0.167573,0.000000,0.000000,0.000000,0.000000,0.009191,0.053475,...,0.000000,0.009711,0.006508,0.000000,0.0,0.000000,0.0,0.02841,0.0,0.000000,0.000000,0.008870,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.033246,0.0,0.000000,0.0000,0.000000,0.000000,0.034092,0.018758,0.000000,0.037930,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.009646
1,0.000000,1.000000,0.000000,0.017839,0.007968,0.000000,0.0,0.012501,0.0,0.014840,0.000000,0.000000,0.000000,0.000000,0.012814,0.000000,0.000000,0.024144,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.008101,0.000000,0.000000,0.016898,0.0,0.017789,0.0,0.008885,0.009432,0.000000,0.000000,0.014947,0.000000,0.000000,0.000000,0.022738,...,0.000000,0.000000,0.019783,0.000000,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.011407,0.011409,0.000000,0.011632,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.015329,0.0,0.008367,0.0,0.000000,0.0000,0.021596,0.000000,0.000000,0.000000,0.000000,0.017564,0.0,0.019152,0.000000,0.000000,0.000000,0.000000,0.007963
2,0.000000,0.000000,1.000000,0.000000,0.017178,0.000000,0.0,0.000000,0.0,0.024326,0.005471,0.018038,0.000000,0.000000,0.000000,0.005099,0.004985,0.004705,0.000000,0.004843,0.000000,0.017026,0.000000,0.003672,0.003984,0.030998,0.006959,0.000000,0.0,0.006117,0.0,0.019548,0.020365,0.009213,0.028467,0.010515,0.004198,0.006311,0.015154,0.012698,...,0.020597,0.004723,0.000000,0.016673,0.0,0.000000,0.0,0.00000,0.0,0.006625,0.000000,0.000000,0.006972,0.000000,0.010574,0.0,0.008222,0.008604,0.012782,0.015353,0.006259,0.000000,0.0,0.000000,0.0,0.010555,0.0000,0.000000,0.000000,0.000000,0.000000,0.006903,0.005024,0.0,0.012893,0.000000,0.025975,0.000000,0.027126,0.009340
3,0.000000,0.017839,0.000000,1.000000,0.000000,0.022414,0.0,0.000000,0.0,0.037207,0.000000,0.000000,0.000000,0.000000,0.027958,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012232,0.017893,0.043639,0.000000,0.000000,0.000000,0.0,0.023127,0.0,0.000000,0.000000,0.000000,0.000000,0.009324,0.022995,0.000000,0.000000,0.044476,...,0.000000,0.000000,0.014029,0.030562,0.0,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.056761,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0000,0.018486,0.000000,0.000000,0.000000,0.060846,0.025039,0.0,0.036237,0.030516,0.022605,0.000000,0.000000,0.000000
4,0.006071,0.007968,0.017178,0.000000,1.000000,0.004673,0.0,0.064581,0.0,0.000000,0.000000,0.036616,0.023883,0.014423,0.012317,0.019837,0.008171,0.004308,0.025463,0.000000,0.008909,0.023739,0.011343,0.003009,0.000000,0.026932,0.000000,0.010027,0.0,0.019560,0.0,0.028525,0.016020,0.011313,0.016914,0.022977,0.054149,0.013769,0.027401,0.005417,...,0.005089,0.015099,0.030763,0.013665,0.0,0.012626,0.0,0.00000,0.0,0.000000,0.027022,0.012027,0.005714,0.000000,0.017233,0.0,0.022330,0.007052,0.038019,0.011785,0.018614,0.021471,0.0,0.003768,0.0,0.008651,0.0095,0.000000,0.012751,0.000000,0.022064,0.019662,0.036561,0.0,0.015826,0.000000,0.076033,0.004516,0.043475,0.011465


In [10]:
movies_list = df['title'].values
movies_list,movies_list.shape

(array(['Minions', 'Interstellar', 'Deadpool', ..., 'Penitentiary',
        'Alien Zone', 'America Is Still the Place'], dtype=object), (4800,))

In [12]:
movie_idx = np.where(movies_list == 'Minions')[0][0]
movie_idx

0

In [13]:
movie_similarities = doc_sim_df.iloc[movie_idx].values
movie_similarities

array([1.        , 0.        , 0.        , ..., 0.        , 0.        ,
       0.00964646])

In [14]:
similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
similar_movie_idxs

array([ 33,  60, 737, 490, 298])

In [15]:
similar_movies = movies_list[similar_movie_idxs]
similar_movies

array(['Despicable Me 2', 'Despicable Me',
       'Teenage Mutant Ninja Turtles: Out of the Shadows', 'Superman',
       'Rise of the Guardians'], dtype=object)

In [16]:
def movie_recommender(movie_title, movies = movies_list, doc_sims = doc_sim_df):
  #find movie id 
  movie_idx = np.where(movies_list == movie_title)[0][0]
  #get movie similarity
  movie_similarities = doc_sim_df.iloc[movie_idx].values
  #get top5 similar movie ID's 
  similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
  #get the movies
  similar_movies = movies_list[similar_movie_idxs]
  return similar_movies

In [17]:
movie_recommender('Interstellar')

array(['Gattaca', 'Space Pirate Captain Harlock', 'Space Cowboys',
       'Starship Troopers', 'Final Destination 2'], dtype=object)

In [18]:
movie_recommender('Terminator Salvation')

array(['Terminator Genisys', 'Terminator 3: Rise of the Machines',
       'Terminator 2: Judgment Day',
       'The Boondock Saints II: All Saints Day', 'The Matador'],
      dtype=object)

In [ ]:
#Fast text to generate the doc sims
from gensim.models import FastText

tokenized_docs = [doc.split() for doc in norm_corpus]
ft_model = FastText(tokenized_docs, size = 300,min_count = 2,window= 30,workers =4,sg = 1, iter=50)

In [ ]:
def averaged_word2vec_vectorizer(corpus, model, num_features):
    vocabulary = set(model.wv.index2word)
    
    def average_word_vectors(words, model, vocabulary, num_features):
        feature_vector = np.zeros((num_features,), dtype="float64")
        nwords = 0.
        
        for word in words:
            if word in vocabulary: 
                nwords = nwords + 1.
                feature_vector = np.add(feature_vector, model.wv[word])
        if nwords:
            feature_vector = np.divide(feature_vector, nwords)

        return feature_vector

    features = [average_word_vectors(tokenized_sentence, model, vocabulary, num_features)
                    for tokenized_sentence in corpus]
    return np.array(features)

In [ ]:
doc_vec_ft = averaged_word2vec_vectorizer(tokenized_docs,ft_model,300)
doc_vec_ft.shape

In [ ]:
#pairwise Document similarity 
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(doc_vec_ft)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

In [ ]:
def movie_recommender(movie_title, movies = movies_list, doc_sims = doc_sim_df):
  #find movie id 
  movie_idx = np.where(movies_list == movie_title)[0][0]
  #get movie similarity
  movie_similarities = doc_sim_df.iloc[movie_idx].values
  #get top5 similar movie ID's 
  similar_movie_idxs = np.argsort(-movie_similarities)[1:6]
  #get the movies
  similar_movies = movies_list[similar_movie_idxs]
  return similar_movies

In [ ]:
movie_recommender('Interstellar')